In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, lower, trim, udf
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql.functions import col, sum, when
from delta import *
import unicodedata

# Caminho do warehouse Hive no HDFS
warehouse_location = "hdfs://hdfs-nn:9000/warehouse"

# Criação da sessão Spark com suporte a Hive + Delta Lake
spark = (
    SparkSession.builder
    .appName("Silver_BoxOffice_Treatment")
    # ---- configurações Hive ----
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.catalogImplementation", "hive")
    .config("hive.metastore.warehouse.dir", warehouse_location)
    # ---- extensões Delta Lake ----
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # ---- pacote Delta compatível com Spark 3.4.1 (Scala 2.12) ----
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .enableHiveSupport()
    .getOrCreate()
)
print("Spark iniciado com sucesso — versão:", spark.version)

Spark iniciado com sucesso — versão: 3.4.1


In [38]:
print("\n 1 — Contagem de NULOS (por coluna):")

spark.table("silver.boxoffice").select(
    sum(when(col("year").isNull(), 1).otherwise(0)).alias("null_year"),
    sum(when(col("title").isNull(), 1).otherwise(0)).alias("null_title"),
    sum(when(col("gross").isNull(), 1).otherwise(0)).alias("null_gross"),
    sum(when(col("decade").isNull(), 1).otherwise(0)).alias("null_decade")
).show()

print("Confirma-se que não existem valores nulos em year/title/gross/decade.")



 1 — Contagem de NULOS (por coluna):
+---------+----------+----------+-----------+
|null_year|null_title|null_gross|null_decade|
+---------+----------+----------+-----------+
|        0|         0|         0|          0|
+---------+----------+----------+-----------+

Confirma-se que não existem valores nulos em year/title/gross/decade.


In [39]:
print("\n 2 — Valores VAZIOS ou INVÁLIDOS:")

spark.table("silver.boxoffice").select(
    sum(when(trim(col("title")) == "", 1).otherwise(0)).alias("title_vazio"),
    sum(when(col("gross") <= 0, 1).otherwise(0)).alias("gross_invalido"),
    sum(when((col("year") < 1900) | (col("year") > 2024), 1).otherwise(0)).alias("anos_invalidos")
).show()

print("Confirma-se que todos estes valores são ZERO -> o dataset Silver está consistente.")



 2 — Valores VAZIOS ou INVÁLIDOS:
+-----------+--------------+--------------+
|title_vazio|gross_invalido|anos_invalidos|
+-----------+--------------+--------------+
|          0|             0|             0|
+-----------+--------------+--------------+

Confirma-se que todos estes valores são ZERO -> o dataset Silver está consistente.


In [40]:
print("\n 3 — Verificação de DUPLICADOS (year + title):")

dups = (
    spark.table("silver.boxoffice")
        .groupBy("year", "title").count()
        .filter(col("count") > 1)
)

dups_count = dups.count()
print(f"\nNúmero de duplicados: {dups_count}")

if dups_count > 0:
    print("Existem duplicados! Mostrar exemplos:")
    dups.show(20, truncate=False)
else:
    print("Confirma-se que não existem valores duplicados -> chave year + title é única.")



 3 — Verificação de DUPLICADOS (year + title):

Número de duplicados: 0
Confirma-se que não existem valores duplicados -> chave year + title é única.


In [41]:
print("\n 4 — Testar Normalização dos Títulos:")

df = spark.table("silver.boxoffice")

print("\nExemplos de títulos normalizados:")
df.select("title") \
  .limit(20).show(truncate=False)

# Verificar se title contém SOMENTE letras minúsculas, números e espaços
invalid = df.filter(~col("title").rlike("^[a-z0-9 ]+$"))

invalid_count = invalid.count()
if invalid_count == 0:
    print("Normalização perfeita -> nenhum caractere especial encontrado.")
else:
    print(f"Foram encontrados {invalid_count} títulos mal normalizados:")
    invalid.show(20, truncate=False)



 4 — Testar Normalização dos Títulos:

Exemplos de títulos normalizados:
+----------------------------+
|title                       |
+----------------------------+
|the flamingo kid            |
|body double                 |
|cross my heart              |
|caddyshack ii               |
|romero                      |
|night of the living dead    |
|welcome home roxy carmichael|
|hidden agenda               |
|pure luck                   |
|tous les matins du monde    |
|shadows and fog             |
|casablanca 1992 rerelease   |
|double dragon               |
|where the rivers flow north |
|leaving las vegas           |
|home for the holidays       |
|bogus                       |
|sprung                      |
|my giant                    |
|the butcher boy             |
+----------------------------+

Normalização perfeita -> nenhum caractere especial encontrado.


In [42]:
print("\n 5 — Coerência da coluna 'decade':")

df_decade = spark.table("silver.boxoffice").select("year", "decade")

# década deve ser sempre um múltiplo de 10
invalid_decades = df_decade.filter((col("decade") % 10) != 0)

if invalid_decades.count() == 0:
    print(" Todas as décadas são válidas.")
else:
    print(" Décadas inválidas encontradas:")
    invalid_decades.show()



 5 — Coerência da coluna 'decade':
 Todas as décadas são válidas.


In [43]:
print("\n 6 — Estatísticas descritivas de 'gross':")

spark.table("silver.boxoffice").describe("gross").show()



 6 — Estatísticas descritivas de 'gross':
+-------+-------------------+
|summary|              gross|
+-------+-------------------+
|  count|               8144|
|   mean| 8.21748439771611E7|
| stddev|1.651777551451544E8|
|    min|             2357.0|
|    max|        2.7994391E9|
+-------+-------------------+



In [44]:
print("\n 7 — Top 10 filmes com maior bilheteira:")

spark.sql("""
    SELECT title, year, gross
    FROM silver.boxoffice
    ORDER BY gross DESC
    LIMIT 10
""").show(truncate=False)



 7 — Top 10 filmes com maior bilheteira:
+----------------------------------------+----+-------------+
|title                                   |year|gross        |
+----------------------------------------+----+-------------+
|avengers endgame                        |2019|2.7994391E9  |
|avatar                                  |2009|2.743577587E9|
|avatar the way of water                 |2022|2.320250281E9|
|star wars episode vii  the force awakens|2015|2.068223624E9|
|avengers infinity war                   |2018|2.048359754E9|
|spiderman no way home                   |2021|1.912233593E9|
|titanic                                 |1997|1.843373318E9|
|jurassic world                          |2015|1.670400637E9|
|the lion king                           |2019|1.656943394E9|
|the avengers                            |2012|1.518812988E9|
+----------------------------------------+----+-------------+



In [45]:
# Quantidade final de linhas da camada Silver
total = spark.table("silver.boxoffice").count()
print(f"\n 8 — Contagem final de registos Silver: {total}")



 8 — Contagem final de registos Silver: 8144


In [46]:
print("\n 9 — Schema Final da tabela Silver:")

spark.sql("DESCRIBE TABLE silver.boxoffice").show(truncate=False)



 9 — Schema Final da tabela Silver:
+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|year    |int      |null   |
|title   |string   |null   |
|gross   |double   |null   |
|decade  |int      |null   |
+--------+---------+-------+



In [47]:
spark.stop()